## Testing app integration

This is an example of using an api endpoint for completions and mocking out the JWT auth

## test with `dispatcher`

### Setup
```bash
git clone https://github.com/Rational-Exponent/dispatcher.git lib/dispatcher
cd lib/dispatcher
pip install -e .
```

In [ ]:
import sys, os
lib_dir = os.path.abspath(os.path.join(os.path.curdir, "lib/dispatcher/src"))
if lib_dir not in sys.path:
    print(f"Adding {lib_dir} to sys.path")
    sys.path.insert(0, lib_dir)
else:
    print(f"{lib_dir} already in sys.path")

os.environ["APP_CONFIG_DIR"] = os.path.abspath(os.path.join(os.path.curdir, "lib/dispatcher/config"))

## Database connection

In [2]:
import sqlite3
# Create a connection to the database
db_connection = sqlite3.connect('../evals.db')

## Integrate with the application and run completions

In [ ]:
from fastapi.testclient import TestClient
from httpx import AsyncClient

# Import our application code to test
# We will test the top-level API application here
from dispatcher.main import api
from dispatcher.models import (
    CreateChatCompletionRequest,
    ChatCompletionRequestUserMessage
)

# Attach a test client to our imported FastAPI application
client = AsyncClient(app=api, base_url="http://test") # TestClient(api)
 
request_payload = {
    "model": "passthrough", 
    "messages": [
        {
            "role": "user",
            "content": "Hello, my world!"
        }
    ],
    "stream": "false" 
}

# We need to patch the `validate_jwt` function to avoid authentication failure
from unittest.mock import patch
with patch("dispatcher.main.validate_jwt", return_value={"user": "test_user", "email": "test@test.com"}):
    response = await client.post(
        "/v1/chat/completions",
        json=request_payload,
        headers={"Authorization": "Bearer test_token"},
    )
    print(">> API RESPONSE: ", response)
    message = response.json()["choices"][0]["message"]["content"]
    print(">> LLM RSESPONSE", message)
    

## test with `mvp` / `chat-service`

### Setup
First we need to install the package into our lab environment

In this experiment folder terminal run:
```bash
git clone https://github.com/Rational-Exponent/mvp.git lib/mvp
cd lib/mvp/app/services/chat
poetry install
```

### Paths

We must add the module code paths to our system so it can import dependencies correctly

In [ ]:
import sys, os
lib_path = os.path.abspath(os.path.join(os.path.curdir, "lib/mvp/app/services/chat"))
if lib_path not in sys.path:
    print(f"Adding {lib_path} to sys.path")
    sys.path.append(lib_path)
else:
    print(f"{lib_path} already in sys.path")

# Add sample data to qdrant

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, Distance
import numpy as np

client = QdrantClient("http://localhost:6333")

collection_name = "test_collection"

if collection_name in client.get_collections():
    client.delete_collection(collection_name)
    print(f"Collection '{collection_name}' deleted.")

dim = 1536
client.recreate_collection(
    collection_name=collection_name,
    vectors_config={
        "size": dim,  # Adjust this to your vector dimension
        "distance": Distance.COSINE
    }
)
print(f"Collection '{collection_name}' created.")

num_vectors = 1  # Number of vectors to generate
vectors = [np.random.rand(dim).tolist() for _ in range(num_vectors)]  # Random vectors

points = [
    PointStruct(
        id=i + 1,  # Unique ID for each point
        vector=vectors[i], 
        payload={"name": f"example_{i + 1}", "category": "test"}
    )
    for i in range(num_vectors)
]

client.upsert(collection_name=collection_name, points=points)
print(f"Points inserted into '{collection_name}'.")


### Running
Run the following to generate a test completion

In [ ]:
from lib.mvp.app.services.chat.main import chat_kb
from lib.mvp.app.services.chat.model import ChatCompletionRequest

request = ChatCompletionRequest(
    model="passthrough",
    messages=[
        dict(
            role="user",
            content="Hello, my world!"
        )
    ],
    stream=False
)

kb = "test_collection"
response = chat_kb(kb, request)
print(response)

# ISSUES

1. The above will connect to local `qdrant` with no data
2. The above will attempt to perform an embedding call to bedrock

In [ ]:
! which pip
! pip list | grep llama